# Objetivo:
- Plotar ocorrências de focos de calor fornecidos pelo INPE num mapa interativo usando o [Leafmap](https://leafmap.org/)
- O `leafmap` é um pacote relativamente novo, versátil e amigável para mapeamento interativo de dados em Python. Desenvolvido por Qiusheng Wu como um derivado do pacote `geemap`, usado exclusivamente no contexto do **Google Earth Engine**, o `leafmap` oferece uma sintaxe simples que permite a criação de mapas interativos de forma eficiente através da integração com outros pacotes como `Folium` e `ipyleaflet`. Antes de mais nada, vamos importá-lo:

# Acesso aos dados:
 - https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/

# Carregando os dados

In [6]:
%%time
# Importa bibliotecas
from datetime import datetime, timedelta, date
import datetime
import pandas as pd

# URL dos dados do INPE
url_diario = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/diario/Brasil/'

# Loop nos dias
df = pd.DataFrame()
anos, meses, dias = [], [], []
for i in range(8):

    # gera as datas
    data = datetime.datetime.now() - timedelta(days=i)
    ano, mes, dia = date.strftime(data,'%Y'), date.strftime(data,'%m'), date.strftime(data,'%d')
    anos.append(ano)
    meses.append(mes)
    dias.append(dia)

    # leitura dos dados
    df0 = pd.read_csv(f'{url_diario}focos_diario_br_{ano}{mes}{dia}.csv')

    # junta a tabela que foi lida com a anterior
    df = pd.concat([df, df0], ignore_index=True)

# seleciona os focos de Minas Gerais
df_mg = df[df['estado']=='MINAS GERAIS']

# mostra os dados
df_mg 

CPU times: total: 156 ms
Wall time: 2.8 s


,id,lat,lon,data_hora_gmt,satelite,municipio,estado,pais,municipio_id,estado_id,pais_id,numero_dias_sem_chuva,precipitacao,risco_fogo,bioma,frp
10,abddc6e7-ab30-37f4-80cc-81de582e57ab,-20.395599,-46.677799,2024-07-05 00:23:00,METOP-C,DELFINÓPOLIS,MINAS GERAIS,Brasil,3121209,31,33,NaN,NaN,NaN,Cerrado,NaN
11,d4104f75-5049-3469-a649-ed359e3729e7,-20.393101,-46.664101,2024-07-05 00:23:00,METOP-C,DELFINÓPOLIS,MINAS GERAIS,Brasil,3121209,31,33,NaN,NaN,NaN,Cerrado,NaN
12,fd5370a1-8fbd-3672-a498-344f948378b6,-20.383400,-46.666000,2024-07-05 00:23:01,METOP-C,DELFINÓPOLIS,MINAS GERAIS,Brasil,3121209,31,33,NaN,NaN,NaN,Cerrado,NaN
13,5394b471-3c75-3b4b-abd0-197ba7ebfe1b,-20.385900,-46.679699,2024-07-05 00:23:01,METOP-C,DELFINÓPOLIS,MINAS GERAIS,Brasil,3121209,31,33,NaN,NaN,NaN,Cerrado,NaN
17,13ad3e5c-6ad4-3a27-b4b0-d8cb68065742,-18.895500,-46.078499,2024-07-05 00:23:23,METOP-C,CARMO DO PARANAÍBA,MINAS GERAIS,Brasil,3114303,31,33,NaN,NaN,NaN,Cerrado,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66022,e7983498-8417-3157-b2f1-1c79559ae19f,-18.892700,-47.501700,2024-06-28 21:46:51,GOES-16,ROMARIA,MINAS GERAIS,Brasil,3156403,31,33,NaN,NaN,NaN,Cerrado,68.4
66023,812f90a3-cf28-34e8-b0ef-ba0ce71e2036,-18.893300,-47.478500,2024-06-28 21:46:51,GOES-16,ROMARIA,MINAS GERAIS,Brasil,3156403,31,33,NaN,NaN,NaN,Cerrado,55.3
66067,e9fcb5c7-51b4-3602-a38a-70b711294685,-16.520300,-43.726600,2024-06-28 21:56:38,GOES-16,FRANCISCO SÁ,MINAS GERAIS,Brasil,3126703,31,33,NaN,NaN,NaN,Cerrado,45.5
66068,c949d4b2-071e-3845-b573-04502833018f,-16.519700,-43.750900,2024-06-28 21:56:38,GOES-16,FRANCISCO SÁ,MINAS GERAIS,Brasil,3126703,31,33,NaN,NaN,NaN,Cerrado,40.6


# Mapa: em forma de `Pontos`

In [7]:
# importa biblioteca
import leafmap.leafmap as leafmap

# configuração do mapa
Map = leafmap.Map(center=(-15, -60), zoom=4)

# adiciona mapa de fundo
Map.add_basemap('HYBRID')

# plota os dados
Map.add_xy_data(df_mg, x="lon", y="lat")

# adiciona contorno dos municípios do Brasil no mapa
shp_mg = 'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-31-mun.json'
Map.add_geojson(shp_mg, layer_name="Munícipios de MG")

# exibe o mapa
Map

Map(center=[-15, -60], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

# Mapa: em forma de `Markers`

In [9]:
# importa biblioteca
import leafmap.leafmap as leafmap

# configuração do mapa
Map = leafmap.Map(center=(-15, -60), zoom=4)

# adiciona mapa de fundo
Map.add_basemap('HYBRID')

# plota os dados
Map.add_circle_markers_from_xy(df_mg,
                               x="lon",
                               y="lat", 
                               radius=2, 
                               color="red",
                               fill=True,
                               fill_color="red",
                               fill_opacity=1)

# adiciona contorno dos estados
shp_estados_brasil = 'https://github.com/codeforgermany/click_that_hood/blob/main/public/data/brazil-states.geojson'
Map.add_geojson(shp_estados_brasil, layer_name="Estados do Brasil")

# exibe o mapa
Map

Map(center=[-15, -60], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

# Mapa: em forma de `Heatmap`

Leitura do dado anual do INPE

In [18]:
%%time
# Importa bibliotecas
from datetime import datetime, timedelta, date
import datetime
import pandas as pd

# URL dos dados mensais do INPE
url_mensal = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/anual/Brasil_sat_ref/'

# nome do arquivo
filename = 'focos_br_ref_2020.zip'

# leitura dos dados
df = pd.read_csv(f'{url_mensal}{filename}', compression='zip', usecols=['lat','lon','data_pas'])

# renomeia algumas colunas
df.rename(columns={'lat': 'latitude', 'lon': 'longitude'}, inplace=True)

# cria uma coluna como valor um para cada ocorrência de foco de calor
df['count'] = 1

# mostra os dados
df

CPU times: total: 156 ms
Wall time: 1.9 s


,latitude,longitude,data_pas,count
0,-3.650,-54.732,2020-06-16 17:00:00,1
1,-4.128,-47.074,2020-06-16 17:00:00,1
2,-10.513,-53.483,2020-06-16 16:55:00,1
3,-10.504,-53.498,2020-06-16 16:55:00,1
4,-10.494,-46.488,2020-06-16 16:55:00,1
...,...,...,...,...
222792,-23.163,-49.164,2020-10-02 17:15:00,1
222793,-9.558,-66.777,2020-10-13 18:40:00,1
222794,-21.691,-44.016,2020-10-13 17:00:00,1
222795,-16.451,-46.062,2020-10-13 17:00:00,1


Plota mapa

In [20]:
# importa biblioteca
import leafmap.leafmap as leafmap

# configuração do mapa
Map = leafmap.Map(center=(-15, -60), zoom=4)

# adiciona mapa de fundo
Map.add_basemap('HYBRID')

# plota os dados
Map.add_heatmap(df,
                x="longitude",
                y="latitude",
                value="count",
                name="Heat map",
                radius=20)

# adiciona contorno dos estados
shp_estados_brasil = 'https://github.com/codeforgermany/click_that_hood/blob/main/public/data/brazil-states.geojson'
Map.add_geojson(shp_estados_brasil, layer_name="Estados do Brasil")

# exibe o mapa
Map

Map(center=[-15, -60], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…